In [ ]:
import time
import eikon as ek
def fetch_data_in_batches(tickers, batch_size, file_output_name, year=None):
    if year is None:
        year = "FY0"
    else:
        year = "FY" + str(year)

    def get_data_batch(batch):
        try:
            data, _ = ek.get_data(
                batch,
                fields=[
                    f'TR.F.BUSTOTREVBIZACTIV(SDate={year}).segmentName', f'TR.F.BUSTOTREVBIZACTIV(SDate={year}).segmentCode'
                ]
            )
            #add a column with the year
            data["year"] = year
            return data  # Return only the DataFrame
        except Exception as e:
            print(f"Failed to fetch data for batch: {batch}. Error: {e}")
            return None  # Return None in case of failure
        
    all_data_frames = []
    batches_to_process = [tickers[i:i + batch_size] for i in range(0, len(tickers), batch_size)]
    failed = []

    while batches_to_process:
        print("Number of batches to process:", len(batches_to_process)+1)
        new_failed = []
        for batch in batches_to_process:
            print("Getting data for batch number:", batches_to_process.index(batch)+1, "of", len(batches_to_process)+1)
            time.sleep(0.2)
            print("Processing batch")
            batch_data = get_data_batch(batch)
            
            if batch_data is not None:
                print("batch successful")
                all_data_frames.append(batch_data)
            else:
                print("batch failed")
                new_failed.append(batch)

        # Update the list of batches to process
        batches_to_process = new_failed

        # Remember the failed batches
        failed.extend(new_failed)

    # Concatenate all the individual batch data frames into one
    final_data_frame = pd.concat(all_data_frames, ignore_index=True) if all_data_frames else pd.DataFrame()

    # Save the data to a file (adjust the path and file name as needed)
    final_data_frame.to_csv(file_output_name, index=False)
    
    return final_data_frame, failed

In [ ]:
import eikon as ek 
ek.set_app_key("5f62315c837645a78ff033f35644b3ba3ef5753a")
import pandas as pd 

relevant_companies = pd.read_csv(r"list_of_non_ren_companies.csv")
for i in range(7): 
    success, fail = fetch_data_in_batches(list(relevant_companies['Instrument']), 200, rf"segment_numbers\no_ren_segment_numbers_for_time_{-i}y.csv", -i)
    segments_relevant_grouped = success.groupby('Instrument').count()
    print(segments_relevant_grouped['Segment Name'].mean())

In [27]:

segment_numbers = {}

segment_codes = ['221111','221113','221114','221115','221116',"221117", '221118']  # Example segment codes
strings_to_check = ['renewable', ' ren ']  

# Creating a regular expression pattern from strings_to_check
pattern = '|'.join(strings_to_check)

for i in range(7): 
    print(i)
    segment_numbers[i] = pd.read_csv(rf"segment_numbers/no_ren_segment_numbers_for_time_{-i}y.csv")
    segment_numbers[i]["year"] = -i

    segment_numbers[i]["number of segments"] = segment_numbers[i].groupby("Instrument")["Segment Name"].transform("count")
    #drop Segment Name	Segment Code





0
1
2
3
4
5
6


In [28]:
for i in range(7):
    segment_numbers[i].to_csv(rf"segment_numbers/no_ren_segment_numbers_for_time_{-i}y.csv", index=False)